In [ ]:
# %pip install optuna

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import necessary libraries
import optuna
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the Pima Indian Diabetes dataset from sklearn
# Note: Scikit-learn's built-in 'load_diabetes' is a regression dataset.
# We will load the actual diabetes dataset from an external source
import pandas as pd

# Load the Pima Indian Diabetes dataset (from UCI repository)
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI',
           'DiabetesPedigreeFunction', 'Age', 'Outcome']

# Load the dataset
df = pd.read_csv(url, names=columns)

df.head()

d:\Programming\Learn_Pytorch_for_deep_learning\.env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
import numpy as np

# Replace zero values with NaN in columns where zero is not a valid value
cols_with_missing_vals = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
df[cols_with_missing_vals] = df[cols_with_missing_vals].replace(0, np.nan)

# Impute the missing values with the mean of the respective column
df.fillna(df.mean(), inplace=True)

# Check if there are any remaining missing values
print(df.isnull().sum())


Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


In [4]:
# Split into features (X) and target (y)
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# Split data into training and test sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Optional: Scale the data for better model performance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Check the shape of the data
print(f'Training set shape: {X_train.shape}')
print(f'Test set shape: {X_test.shape}')


Training set shape: (537, 8)
Test set shape: (231, 8)


In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize


In [6]:
# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters


[I 2025-09-15 21:53:18,372] A new study created in memory with name: no-name-37519ad7-2d15-4a8c-888a-8f2c9e4c84cb
[I 2025-09-15 21:53:18,681] Trial 0 finished with value: 0.7746741154562383 and parameters: {'n_estimators': 130, 'max_depth': 12}. Best is trial 0 with value: 0.7746741154562383.
[I 2025-09-15 21:53:19,106] Trial 1 finished with value: 0.7746741154562384 and parameters: {'n_estimators': 185, 'max_depth': 19}. Best is trial 1 with value: 0.7746741154562384.
[I 2025-09-15 21:53:19,283] Trial 2 finished with value: 0.7523277467411545 and parameters: {'n_estimators': 89, 'max_depth': 3}. Best is trial 1 with value: 0.7746741154562384.
[I 2025-09-15 21:53:19,511] Trial 3 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 98, 'max_depth': 7}. Best is trial 1 with value: 0.7746741154562384.
[I 2025-09-15 21:53:19,909] Trial 4 finished with value: 0.756052141527002 and parameters: {'n_estimators': 190, 'max_depth': 6}. Best is trial 1 with value: 0.7746741154

In [7]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7821229050279329
Best hyperparameters: {'n_estimators': 119, 'max_depth': 19}


In [8]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.74


## Samplers in Optuna

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize


In [10]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters

[I 2025-09-15 21:53:32,671] A new study created in memory with name: no-name-aadec10d-d05c-4699-88b8-a8c1107ae795
[I 2025-09-15 21:53:33,044] Trial 0 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 165, 'max_depth': 11}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-09-15 21:53:33,161] Trial 1 finished with value: 0.7597765363128491 and parameters: {'n_estimators': 56, 'max_depth': 3}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-09-15 21:53:33,388] Trial 2 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 97, 'max_depth': 10}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-09-15 21:53:33,638] Trial 3 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 93, 'max_depth': 16}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-09-15 21:53:33,827] Trial 4 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 86, 'max_depth': 5}. Best is trial 0 with value: 0.7690875232

In [11]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7783985102420856
Best hyperparameters: {'n_estimators': 132, 'max_depth': 13}


In [12]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.75


In [13]:
search_space = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [5, 10, 15, 20]
}

In [14]:
# Create a study and optimize it using GridSampler
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.GridSampler(search_space))
study.optimize(objective)

[I 2025-09-15 21:53:46,031] A new study created in memory with name: no-name-cb4926ad-a572-4ed7-b82c-31fe1524816f
[I 2025-09-15 21:53:46,239] Trial 0 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 5}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-09-15 21:53:46,576] Trial 1 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 150, 'max_depth': 10}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-09-15 21:53:46,694] Trial 2 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 50, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-09-15 21:53:46,926] Trial 3 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 100, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-09-15 21:53:47,152] Trial 4 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 20}. Best is trial 2 with value: 0.772811

In [15]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7746741154562384
Best hyperparameters: {'n_estimators': 50, 'max_depth': 5}


In [16]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.74


## Optuna Visualizations

In [17]:
# For visualizations
from optuna.visualization import plot_optimization_history, plot_parallel_coordinate, plot_slice, plot_contour, plot_param_importances

In [18]:
# 1. Optimization History
plot_optimization_history(study).show()

In [19]:
# 2. Parallel Coordinates Plot
plot_parallel_coordinate(study).show()

In [20]:
# 3. Slice Plot
plot_slice(study).show()

In [21]:
# 4. Contour Plot
plot_contour(study).show()

In [22]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

## Optimizing Multiple ML Models

In [23]:
# Importing the required libraries
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

In [24]:
# Define the objective function for Optuna
def objective(trial):
    # Choose the algorithm to tune
    classifier_name = trial.suggest_categorical('classifier', ['SVM', 'RandomForest', 'GradientBoosting'])

    if classifier_name == 'SVM':
        # SVM hyperparameters
        c = trial.suggest_float('C', 0.1, 100, log=True)
        kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly', 'sigmoid'])
        gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])

        model = SVC(C=c, kernel=kernel, gamma=gamma, random_state=42)

    elif classifier_name == 'RandomForest':
        # Random Forest hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
        bootstrap = trial.suggest_categorical('bootstrap', [True, False])

        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            bootstrap=bootstrap,
            random_state=42
        )

    elif classifier_name == 'GradientBoosting':
        # Gradient Boosting hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

        model = GradientBoostingClassifier(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        )

    # Perform cross-validation and return the mean accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()
    return score

In [25]:
# Create a study and optimize it using CmaEsSampler
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2025-09-15 21:53:53,886] A new study created in memory with name: no-name-51ff3e93-12e5-41f3-9ae7-5d4b2f39ca1b
[I 2025-09-15 21:53:54,900] Trial 0 finished with value: 0.7616387337057727 and parameters: {'classifier': 'GradientBoosting', 'n_estimators': 162, 'learning_rate': 0.024099349174660747, 'max_depth': 14, 'min_samples_split': 9, 'min_samples_leaf': 6}. Best is trial 0 with value: 0.7616387337057727.
[I 2025-09-15 21:53:55,459] Trial 1 finished with value: 0.7523277467411545 and parameters: {'classifier': 'RandomForest', 'n_estimators': 275, 'max_depth': 10, 'min_samples_split': 9, 'min_samples_leaf': 9, 'bootstrap': True}. Best is trial 0 with value: 0.7616387337057727.
[I 2025-09-15 21:53:55,818] Trial 2 finished with value: 0.7579143389199254 and parameters: {'classifier': 'RandomForest', 'n_estimators': 175, 'max_depth': 17, 'min_samples_split': 7, 'min_samples_leaf': 6, 'bootstrap': True}. Best is trial 0 with value: 0.7616387337057727.
[I 2025-09-15 21:53:56,875] Trial 

In [26]:
# Retrieve the best trial
best_trial = study.best_trial
print("Best trial parameters:", best_trial.params)
print("Best trial accuracy:", best_trial.value)

Best trial parameters: {'classifier': 'SVM', 'C': 0.11588247837023241, 'kernel': 'linear', 'gamma': 'auto'}
Best trial accuracy: 0.7895716945996275


In [27]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_C,params_bootstrap,params_classifier,params_gamma,params_kernel,params_learning_rate,params_max_depth,params_min_samples_leaf,params_min_samples_split,params_n_estimators,state
0,0,0.761639,2025-09-15 21:53:53.887926,2025-09-15 21:53:54.900143,0 days 00:00:01.012217,NaN,NaN,GradientBoosting,NaN,NaN,0.024099,14.0,6.0,9.0,162.0,COMPLETE
1,1,0.752328,2025-09-15 21:53:54.901161,2025-09-15 21:53:55.458475,0 days 00:00:00.557314,NaN,True,RandomForest,NaN,NaN,NaN,10.0,9.0,9.0,275.0,COMPLETE
2,2,0.757914,2025-09-15 21:53:55.459978,2025-09-15 21:53:55.818691,0 days 00:00:00.358713,NaN,True,RandomForest,NaN,NaN,NaN,17.0,6.0,7.0,175.0,COMPLETE
3,3,0.748603,2025-09-15 21:53:55.819685,2025-09-15 21:53:56.875399,0 days 00:00:01.055714,NaN,NaN,GradientBoosting,NaN,NaN,0.011234,11.0,5.0,6.0,171.0,COMPLETE
4,4,0.770950,2025-09-15 21:53:56.876405,2025-09-15 21:53:57.130615,0 days 00:00:00.254210,NaN,True,RandomForest,NaN,NaN,NaN,20.0,5.0,2.0,118.0,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,0.789572,2025-09-15 21:54:07.679955,2025-09-15 21:54:07.691624,0 days 00:00:00.011669,0.133325,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
96,96,0.767225,2025-09-15 21:54:07.692624,2025-09-15 21:54:07.705991,0 days 00:00:00.013367,0.110062,NaN,SVM,scale,sigmoid,NaN,NaN,NaN,NaN,NaN,COMPLETE
97,97,0.785847,2025-09-15 21:54:07.707560,2025-09-15 21:54:07.766584,0 days 00:00:00.059024,22.756491,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
98,98,0.785847,2025-09-15 21:54:07.767276,2025-09-15 21:54:07.793600,0 days 00:00:00.026324,4.118575,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE


In [28]:
study.trials_dataframe()['params_classifier'].value_counts()

params_classifier
SVM                 77
RandomForest        12
GradientBoosting    11
Name: count, dtype: int64

In [29]:
study.trials_dataframe().groupby('params_classifier')['value'].mean()

params_classifier
GradientBoosting    0.748265
RandomForest        0.765673
SVM                 0.775182
Name: value, dtype: float64

In [30]:
# 1. Optimization History
plot_optimization_history(study).show()

In [31]:
# 3. Slice Plot
plot_slice(study).show()

In [32]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

In [43]:
# %pip install xgboost

In [44]:
%pip install optuna-integration[xgboost]

Note: you may need to restart the kernel to use updated packages.


In [45]:
import optuna
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
import numpy as np

# Load the Iris dataset
X, y = load_iris(return_X_y=True)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the objective function for XGBoost
def objective(trial):
    # Hyperparameter search space
    param = {
        'verbosity': 0,
        'objective': 'multi:softprob',
        'num_class': 3,
        'eval_metric': 'mlogloss',  # Ensure that the eval_metric is specified here
        'booster': 'gbtree',
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.4, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'n_estimators': 300,
    }

    # Create DMatrix for XGBoost
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)

    # Define a pruning callback based on evaluation metrics
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "eval-mlogloss")  # Match the metric name in the evals list

    # Train the model
    bst = xgb.train(
        param,
        dtrain,
        num_boost_round=300,
        evals=[(dtrain, "train"), (dtest, "eval")],  # Ensure the eval datasets and names are specified
        early_stopping_rounds=30,
        callbacks=[pruning_callback]
    )

    # Predict on the test set
    preds = bst.predict(dtest)
    best_preds = [int(np.argmax(line)) for line in preds]

    # Return accuracy as the objective value
    accuracy = accuracy_score(y_test, best_preds)
    return accuracy

# Create a study with pruning
study = optuna.create_study(direction='maximize', pruner=optuna.pruners.SuccessiveHalvingPruner())
study.optimize(objective, n_trials=50)

# Output the best trial
print(f"Best trial: {study.best_trial.params}")
print(f"Best accuracy: {study.best_value}")


[I 2025-09-15 21:56:57,506] A new study created in memory with name: no-name-1d51bdcc-3ac0-4fd5-8757-528d5980f1fe


[0]	train-mlogloss:0.88371	eval-mlogloss:0.88501
[1]	train-mlogloss:0.78885	eval-mlogloss:0.79316
[2]	train-mlogloss:0.69576	eval-mlogloss:0.71384
[3]	train-mlogloss:0.56692	eval-mlogloss:0.56194
[4]	train-mlogloss:0.52993	eval-mlogloss:0.52775
[5]	train-mlogloss:0.47668	eval-mlogloss:0.47644
[6]	train-mlogloss:0.39684	eval-mlogloss:0.37876
[7]	train-mlogloss:0.33908	eval-mlogloss:0.30967
[8]	train-mlogloss:0.32125	eval-mlogloss:0.29057
[9]	train-mlogloss:0.27128	eval-mlogloss:0.23279
[10]	train-mlogloss:0.24573	eval-mlogloss:0.20074
[11]	train-mlogloss:0.23646	eval-mlogloss:0.19489
[12]	train-mlogloss:0.21236	eval-mlogloss:0.16621
[13]	train-mlogloss:0.19413	eval-mlogloss:0.14627
[14]	train-mlogloss:0.17441	eval-mlogloss:0.12113
[15]	train-mlogloss:0.16700	eval-mlogloss:0.11333
[16]	train-mlogloss:0.15979	eval-mlogloss:0.10655
[17]	train-mlogloss:0.14730	eval-mlogloss:0.09292
[18]	train-mlogloss:0.14294	eval-mlogloss:0.08984
[19]	train-mlogloss:0.14002	eval-mlogloss:0.08817
[20]	train

[I 2025-09-15 21:56:57,820] Trial 0 finished with value: 1.0 and parameters: {'lambda': 1.9818592149450373e-05, 'alpha': 5.050805957577253e-08, 'eta': 0.23071761849846267, 'gamma': 0.6287801412121359, 'max_depth': 8, 'min_child_weight': 1, 'subsample': 0.49704534441995246, 'colsample_bytree': 0.49355529594145847}. Best is trial 0 with value: 1.0.


[0]	train-mlogloss:0.95833	eval-mlogloss:0.95196
[1]	train-mlogloss:0.84262	eval-mlogloss:0.83176
[2]	train-mlogloss:0.74434	eval-mlogloss:0.72797
[3]	train-mlogloss:0.66062	eval-mlogloss:0.63834
[4]	train-mlogloss:0.59141	eval-mlogloss:0.56422
[5]	train-mlogloss:0.53070	eval-mlogloss:0.49959
[6]	train-mlogloss:0.47860	eval-mlogloss:0.44737
[7]	train-mlogloss:0.43129	eval-mlogloss:0.39875
[8]	train-mlogloss:0.39056	eval-mlogloss:0.35617
[9]	train-mlogloss:0.35445	eval-mlogloss:0.31755
[10]	train-mlogloss:0.32173	eval-mlogloss:0.28986
[11]	train-mlogloss:0.29396	eval-mlogloss:0.25895
[12]	train-mlogloss:0.26985	eval-mlogloss:0.23367
[13]	train-mlogloss:0.24774	eval-mlogloss:0.21067
[14]	train-mlogloss:0.22906	eval-mlogloss:0.18977
[15]	train-mlogloss:0.21102	eval-mlogloss:0.17158
[16]	train-mlogloss:0.19548	eval-mlogloss:0.15399
[17]	train-mlogloss:0.17994	eval-mlogloss:0.14298
[18]	train-mlogloss:0.16787	eval-mlogloss:0.13161
[19]	train-mlogloss:0.15556	eval-mlogloss:0.12051
[20]	train

[I 2025-09-15 21:56:58,302] Trial 1 finished with value: 1.0 and parameters: {'lambda': 0.0007615727826442108, 'alpha': 5.2481187542195936e-08, 'eta': 0.10934183215749184, 'gamma': 0.05384165054132559, 'max_depth': 9, 'min_child_weight': 2, 'subsample': 0.808632091458841, 'colsample_bytree': 0.9026618529663907}. Best is trial 0 with value: 1.0.


...

[0]	train-mlogloss:0.77338	eval-mlogloss:0.77094
[1]	train-mlogloss:0.56924	eval-mlogloss:0.54958


[I 2025-09-15 21:57:01,533] Trial 48 pruned. Trial was pruned at iteration 1.


[0]	train-mlogloss:0.97650	eval-mlogloss:0.96934
[1]	train-mlogloss:0.91371	eval-mlogloss:0.90684


[I 2025-09-15 21:57:01,555] Trial 49 pruned. Trial was pruned at iteration 1.


Best trial: {'lambda': 1.9818592149450373e-05, 'alpha': 5.050805957577253e-08, 'eta': 0.23071761849846267, 'gamma': 0.6287801412121359, 'max_depth': 8, 'min_child_weight': 1, 'subsample': 0.49704534441995246, 'colsample_bytree': 0.49355529594145847}
Best accuracy: 1.0


In [48]:
# ! pip install optuna-integration[xgboost]

In [49]:
from optuna.visualization import plot_intermediate_values

# 1. Plot intermediate values during the trials
plot_intermediate_values(study).show()